# Plotting II - due 10/07 *by Charlotte*

---

In [145]:
import pandas as pd

wdi = pd.read_csv(
    "https://raw.githubusercontent.com/nickeubank/practicaldatascience"
    "/master/Example_Data/world-small.csv"
)

In [146]:
wdi.head()

,country,region,gdppcap08,polityIV
0,Albania,C&E Europe,7715,17.8
1,Algeria,Africa,8033,10.0
2,Angola,Africa,5899,8.0
3,Argentina,S. America,14333,18.0
4,Armenia,C&E Europe,6070,15.0


---
### Exercise 1
Let’s being analyzing this data by estimating a simple linear model (“ordinary least squares”) of the relationship between GDP per capita (gdppcap08) and democracy scores (polityIV). We will do so using the statsmodel package, which we’ll discuss in detail later is this course. For the momement, just use this code:

In [147]:
import statsmodels.formula.api as smf
results = smf.ols('polityIV ~ gdppcap08',
                   data=wdi).fit()
print("OLS Model Polity IV regressed on gdppcap08:")
print(results.summary())

OLS Model Polity IV regressed on gdppcap08:
                            OLS Regression Results                            
Dep. Variable:               polityIV   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     6.981
Date:                Sun, 10 Oct 2021   Prob (F-statistic):            0.00915
Time:                        11:29:36   Log-Likelihood:                -475.14
No. Observations:                 145   AIC:                             954.3
Df Residuals:                     143   BIC:                             960.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercep

---
### Exercise 2
Based on the results of this analysis, what would you conclude about about the relationship between gdppcap08 and polityIV?

- **The relationship is marginally positive and significant**: With a one unit increase in the gdppcap08 variable, the polity IV index increases by 9.602e-05, i.e. 0.00009062. In a more understandable interpretation this means that with an increase of $10,000 GDP per cap, the Polity IV Democracy Score would increase by 0.9 units. The coefficient is significant at the 99% level with a p-value of 0.009. 

---
### Exercise 3
Now let’s plot the relationship you just estimated statistically. First, use Altair to create a scatter plot of polityIV and gdppcap08.


In [148]:
import altair as alt
alt.Chart(wdi, title="Relationship between GDP per capita and the Polity IV Score").mark_point().encode(
    x=alt.X("gdppcap08", title="GDP per Capita", scale=alt.Scale(zero=False)),
    y=alt.Y("polityIV", title="Polity IV Democracy Score", scale=alt.Scale(zero=False)),
)


alt.Chart(...)

---
### Exercise 4
Now overlay the linear model you just estimated.

In [149]:
base = (
    alt.Chart(wdi, title="Relationship between GDP per capita and the Polity IV Score")
    .mark_point()
    .encode(
        x=alt.X("gdppcap08", title="GDP per Capita", scale=alt.Scale(zero=False)),
        y=alt.Y("polityIV", title="Polity IV Democracy Score", scale=alt.Scale(zero=False)),
    )
)
fit0 = base.transform_regression(
        "gdppcap08", "polityIV"
    ).mark_line()

text = (alt.Chart(wdi)
    .encode(
        x=alt.X("gdppcap08", scale=alt.Scale(zero=False)),
        y="polityIV",
    )
    .mark_text(size=5))

base + fit0 + text

alt.LayerChart(...)

---
### Exercise 5
Does it seem like the linear model you estimated fits the data well?

- No, the model does not fit our data well. The relationship is not linear.

---
### Exercise 6
Linear models impose a very strict functional form on the model they use: they try to draw a straight line through the data, no matter what.

Can you think of a transform for your data that would make the data a little more sane?

In [150]:
print("We could try to fit a quadratic form:")
import numpy as np
model = np.poly1d(np.polyfit(wdi["gdppcap08"], wdi["polityIV"], 2))

base = (
    alt.Chart(wdi, title="Relationship between GDP per capita and the Polity IV Score")
    .mark_point()
    .encode(
        x=alt.X("gdppcap08", title="GDP per Capita", scale=alt.Scale(zero=False)),
        y=alt.Y("polityIV", title="Polity IV Democracy Score", scale=alt.Scale(zero=False)),
    )
)
fit = base.transform_regression(
        "gdppcap08", "polityIV", 
        method="quad"
    ).mark_line()

text = (alt.Chart(wdi)
    .encode(
        x=alt.X("gdppcap08", scale=alt.Scale(zero=False)),
        y="polityIV",
    )
    .mark_text(size=5))

base + fit + text

We could try to fit a quadratic form:


alt.LayerChart(...)

---
### Exercise 7

Once you’ve applied that transformation, let’s re-fit our model. Rather than imposing linearity this time, however, let’s fit a model with a flexible functional form. Use transform_loess() to fit an updated model over your data. This is a form of local polynomial regression that is designed to be flexible in how it fits the data.

In [151]:
import numpy as np
model = np.poly1d(np.polyfit(wdi["gdppcap08"], wdi["polityIV"], 2))

base = (
    alt.Chart(wdi, title="Relationship between GDP per capita and the Polity IV Score")
    .mark_point()
    .encode(
        x=alt.X("gdppcap08", title="GDP per Capita", scale=alt.Scale(zero=False)),
        y=alt.Y("polityIV", title="Polity IV Democracy Score", scale=alt.Scale(zero=False)),
    )
)
fit2 = base.transform_loess(
        "gdppcap08", "polityIV"
        
    ).mark_line()

text = (alt.Chart(wdi)
    .encode(
        x=alt.X("gdppcap08", scale=alt.Scale(zero=False)),
        y="polityIV",
        text="country"
    )
    .mark_text(size=5))

base + fit2 + text

alt.LayerChart(...)

---
### Exercise 8

This does seem to fit the data better, but there’s clearly this HUGE outlier in the bottom right. Who is that? Add text labels to the points on your graph with country names.

In [152]:
model = np.poly1d(np.polyfit(wdi["gdppcap08"], wdi["polityIV"], 2))

base = (
    alt.Chart(wdi, title="Relationship between GDP per capita and the Polity IV Score")
    .mark_point()
    .encode(
        x=alt.X("gdppcap08", title="GDP per Capita", scale=alt.Scale(zero=False)),
        y=alt.Y("polityIV", title="Polity IV Democracy Score", scale=alt.Scale(zero=False)),
        tooltip="country"
    )
).interactive()


base + fit2 + text

alt.LayerChart(...)

- The outlier is Qatar. Other states that are very low on Polity IV scores but high in GDP per capita are: UAE, Kuwait, Oman and Bahrain. This makes me think of the Resource Curse. 

---
### Exercise 9
Let’s see what happens if we exclude the ten countries with the highest per-capita oil production from our data: Qatar, Kuwait, Equatorial Guinea, United Arab Emirates, Norway, Saudi Arabia, Libya, Oman, Gabon, and Angola.

In [153]:
print("Dropping Outliers")
sub_wdi = wdi[wdi.country != "Qatar" ]
sub_wdi = sub_wdi[sub_wdi.country != "Kuwait"]
sub_wdi = sub_wdi[sub_wdi.country != "Equatorial Guinea"]
sub_wdi = sub_wdi[sub_wdi.country != "UAE"]
sub_wdi = sub_wdi[sub_wdi.country != "Norway"]
sub_wdi = sub_wdi[sub_wdi.country != "Saudi Arabia"]
sub_wdi = sub_wdi[sub_wdi.country != "Libya"]
sub_wdi = sub_wdi[sub_wdi.country != "Oman"]
sub_wdi = sub_wdi[sub_wdi.country != "Gabon"]
sub_wdi = sub_wdi[sub_wdi.country != "Angola"]
sub_wdi


Dropping Outliers


,country,region,gdppcap08,polityIV
0,Albania,C&E Europe,7715,17.8
1,Algeria,Africa,8033,10.0
3,Argentina,S. America,14333,18.0
4,Armenia,C&E Europe,6070,15.0
5,Australia,Asia-Pacific,35677,20.0
...,...,...,...,...
140,Venezuela,S. America,12804,16.0
141,Vietnam,Asia-Pacific,2785,3.0
142,Yemen,Middle East,2400,8.0
143,Zambia,Africa,1356,15.0


What does the relationship between Polity and GDP per capita look like for non-natural resource producers?

In [154]:
base = (
    alt.Chart(sub_wdi, title="Relationship between GDP per capita and the Polity IV Score")
    .mark_point()
    .encode(
        x=alt.X("gdppcap08", title="GDP per Capita", scale=alt.Scale(zero=False)),
        y=alt.Y("polityIV", title="Polity IV Democracy Score", scale=alt.Scale(zero=False)),
        tooltip="country"
).interactive()
)

text = (alt.Chart(sub_wdi)
    .encode(
        x=alt.X("gdppcap08", scale=alt.Scale(zero=False)),
        y="polityIV",
        text="country"
    )
    .mark_text(size=5))

print("Fitting a loess")
base + fit2 + text


Fitting a loess


alt.LayerChart(...)

-The relationship now seems more destinct and rather positive, however still not linear. I assume dropping Singapore and Bahrain would change that. Let's see:

In [155]:
sub2_wdi = sub_wdi[sub_wdi.country != "Singapore"]
sub2_wdi = sub2_wdi[sub_wdi.country != "Bahrain"]
sub2_wdi
base = (
    alt.Chart(sub2_wdi, title="Relationship between GDP per capita and the Polity IV Score")
    .mark_point()
    .encode(
        x=alt.X("gdppcap08", title="GDP per Capita", scale=alt.Scale(zero=False)),
        y=alt.Y("polityIV", title="Polity IV Democracy Score", scale=alt.Scale(zero=False)),
        tooltip="country"
).interactive()
)

text = (alt.Chart(sub2_wdi)
    .encode(
        x=alt.X("gdppcap08", scale=alt.Scale(zero=False)),
        y="polityIV",
        text="country"
    )
    .mark_text(size=5))

base + fit + text

/var/folders/h9/g02cmrsn6y571zb1v96gs56c0000gn/T/ipykernel_77670/3036208200.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sub2_wdi = sub2_wdi[sub_wdi.country != "Bahrain"]


alt.LayerChart(...)

---
### Exercise 10
Let’s make sure that you accurately identified all 10 of the oil producers. Write a line of code to count up how many big producers you have identified. If you do not get 10, can you figure out what you did wrong?

In [156]:
import pandas as pd
oil= pd.concat([wdi,sub_wdi]).drop_duplicates(keep=False)
oil

,country,region,gdppcap08,polityIV
2,Angola,Africa,5899,8.0
39,Equatorial Guinea,Africa,33873,5.0
45,Gabon,Africa,14527,6.0
71,Kuwait,Middle East,39914,3.0
77,Libya,Middle East,15402,3.0
98,Norway,Scandinavia,58138,20.0
99,Oman,Middle East,22478,2.0
107,Qatar,Middle East,85868,0.0
111,Saudi Arabia,Middle East,23920,0.0
133,UAE,Middle East,38830,2.0


I get 10 big producers that I have dropped from the original dataset! So, everything is correct. However, I have noticed before that in the Exercise it said "United Arab Emirates" but in the dataframe the country code is "UAE", so I had dropped it accordingly. 

---
### Exercise 11

How does the relationship between GDP per capita and Polity look for the oil producers we dropped above?


In [157]:
base = (
    alt.Chart(oil, title="Relationship between GDP per capita and the Polity IV Score")
    .mark_point()
    .encode(
        x=alt.X("gdppcap08", title="GDP per Capita", scale=alt.Scale(zero=False)),
        y=alt.Y("polityIV", title="Polity IV Democracy Score", scale=alt.Scale(zero=False)),
        tooltip="country"
).interactive()
)

text = (alt.Chart(oil)
    .encode(
        x=alt.X("gdppcap08", scale=alt.Scale(zero=False)),
        y="polityIV",
        text="country"
    )
    .mark_text(size=5))

base + fit + text

alt.LayerChart(...)

- The relationship is not linear, however, that is because of Norway which is both a very democratic country and a big oil producer. The resource curse does not affect Norwegian scores of democracy. If Norway would be dropped, I'd assume the relationship between GDP per capita and democracy scores would be rather linear and negative.

---
### Exercise 12

Look back to your answer for Exercise 2. Do you still believe the result of your linear model? What did you learn from plotting. Write down your answers with your partner.
- I do not believe the result of the linear model. I fitted a linear function and it forced itself on the data without the data actually showing a linear relationship between the two variables. It teaches me that it is necessary to understand the data first, investigate the dataset, look for potential outliers or data that might skew any substantial findings. It also teaches me that it's important what research question we have at hand and to adjust the model to it. E.g. do we want to see how the resource curse plays out? Then we should keep the outliers and big oil producers. Or do we want to look at countries that are in a certain economic sector? It all matters. Generally, the functional form we apply to our regression needs to be set based on good arguments and according to the data. With all datapoints included the function here e.g. rather seems to be quadratical than linear. OLS does hence not always work or produce results that reflect reality. 

---
### Exercise 13

Finally, let’s make an interactive chart to help readers explore this relationship themselves! Create an interactive chart with a dropdown menu to select big oil producers / non-big oil producers, and add a mouseover to allow users to check the names of countries.

In [162]:
import numpy as np
print("Create Boolean indicator")
wdi["Oil Producer"] = (wdi["country"] == "Qatar") | (wdi["country"] == "Kuwait") |  (wdi["country"] == "Equatorial Guinea")  | (wdi["country"] == "UAE")  | (wdi["country"] == "Norway")  | (wdi["country"] == "Saudi Arabia")  | (wdi["country"] == "Libya")  | (wdi["country"] == "Oman") | (wdi["country"] == "Gabon") | (wdi["country"] =="Angola")
wdi["Oil Producer"]
wdi['Oil Producing'] = [1 if x ==True else 0 for x in wdi['Oil Producer']]
wdi['Oil Producing'] = wdi['Oil Producing'].replace(1 , "Yes").copy()
wdi['Oil Producing'] = wdi['Oil Producing'].replace(0, "No").copy()
wdi['Oil Producing']

Create Boolean indicator


0       No
1       No
2      Yes
3       No
4       No
      ... 
140     No
141     No
142     No
143     No
144     No
Name: Oil Producing, Length: 145, dtype: object

In [166]:
input_dropdown = alt.binding_select(options=['Yes', 'No'])
selection = alt.selection_single(fields=['Oil Producing'], bind=input_dropdown, name='Country is')
color = alt.condition(selection,
                    alt.Color('Oil Producing:N', legend=None),
                    alt.value('lightgray'))

base = (
    alt.Chart(wdi, title="Relationship between GDP per capita and the Polity IV Score")
    .mark_point()
    .encode(
        x=alt.X("gdppcap08", title="GDP per Capita", scale=alt.Scale(zero=False)),
        y=alt.Y("polityIV", title="Polity IV Democracy Score", scale=alt.Scale(zero=False)),
        color=color,
        tooltip="countr:N"
    ).add_selection(
        selection
).interactive()
)

text = (alt.Chart(wdi)
    .encode(
        x=alt.X("gdppcap08", scale=alt.Scale(zero=False)),
        y="polityIV",
        text="country"
    )
    .mark_text(size=5))

base +  text

alt.LayerChart(...)